In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#Importing packages
import tensorflow as tf
import os
import cv2
import imageio
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.patches as mpatches
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten
from tensorflow.keras.models import Model
import random as rn
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import VGG16
import datetime
import glob
import warnings
from tensorflow.keras import models, layers
from sklearn.metrics import cohen_kappa_score
import math
from keras.regularizers import l1 ,l2
import keras
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from matplotlib import pyplot as plt

import argparse
import os
import warnings

from keras.callbacks import Callback
from keras import backend as K
warnings.filterwarnings('ignore')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**1.1 Reading data**

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

img_path = list()
img_fullname = list()
img_name = list()
for dirname, _, filenames in os.walk('/kaggle/input/aptos2019-blindness-detection/train_images/'):
    for filename in filenames:
        img_path.append(os.path.join(dirname, filename))
        temp = os.path.join(dirname, filename)
        temp = temp.split("/")[-1]
        img_fullname.append(temp)
        temp = temp.split(".")[0]
        img_name.append(str(temp))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pre_df = pd.DataFrame()
pre_df['path'] = img_path
pre_df['fullname'] = img_fullname
pre_df['id_code'] = img_name

In [ ]:
# Reading DR grades files of 2019, Messidor & IDRiD
train2019 = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
train2019 = train2019[['id_code', 'diagnosis']]

In [ ]:
# Fetching DR grades of the respective images of 2019
labels = list()
id_code = list()
for i, j in train2019.iterrows():
    id_code.append(j['id_code'])
    labels.append(j['diagnosis'])

In [ ]:
train_labels = pd.DataFrame() # Storing the image file name & DR grades into the dataframe
train_labels['id_code'] = id_code
train_labels['diagnosis'] = labels

In [ ]:
# Merging the image path & DR grades 
pre_df = pd.merge(pre_df, train_labels, on='id_code', how='left')

In [ ]:
pre_df = pre_df[pre_df['diagnosis'].notna()] # removing row if the DR grade is NaN

In [ ]:
print(pre_df[pre_df['diagnosis'].isnull()]) # Verify if there is any NaN present in the DR grades field

In [ ]:
convert_dict = {'diagnosis': str}
  
pre_df = pre_df.astype(convert_dict)

**Train and test data split up**

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
y = pre_df['diagnosis'].values
X_train, X_test, y_train, y_test = train_test_split(pre_df, y, test_size=0.20, stratify=y)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Hyperparameters
batch_size = 8
num_classes = 5
l = 14
num_filter = 32
compression = 0.5
dropout_rate = 0

In [ ]:
img_height = 32
img_width = 32
channel = 3

**Building DensetNet Model**

In [ ]:
tf.keras.backend.clear_session()
# Dense Block
def denseblock(input, num_filter, dropout_rate):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter, dropout_rate):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    flat = layers.Flatten()(AvgPooling)
    output = layers.Dense(num_classes, activation='softmax')(flat)
    return output

In [ ]:
num_filter = 32
dropout_rate = 0
l = 14

tf.keras.backend.clear_session()
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(24, (2,2), use_bias=False ,padding='same', kernel_initializer='he_uniform', bias_initializer='zeros',kernel_regularizer=l2(0.0001))(input)

First_Block = denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = transition(First_Block, num_filter, dropout_rate)

Second_Block = denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = transition(Second_Block, num_filter, dropout_rate)

Third_Block = denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = transition(Third_Block, num_filter, dropout_rate)

Last_Block = denseblock(Third_Transition, num_filter, dropout_rate)
output = output_layer(Last_Block)

In [ ]:
model = Model(inputs=[input], outputs=[output])
model.summary()

In [ ]:
#optimizer = SGD(learning_rate=0.1, momentum=0.9, decay= 1e-6, nesterov=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
# reference URL - https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
from keras_preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(rescale=1./255., validation_split=0.20, horizontal_flip=True)

train_generator=datagen.flow_from_dataframe(dataframe=X_train, directory=None, x_col="path", y_col="diagnosis", subset="training", 
                batch_size=8, shuffle=True, class_mode="categorical", drop_duplicates = False, target_size=(32,32))

valid_generator=datagen.flow_from_dataframe(dataframe=X_train, directory=None, x_col="path", y_col="diagnosis", subset="validation", 
                        batch_size=8, shuffle=True, class_mode="categorical", drop_duplicates = False, target_size=(32,32))

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(dataframe=X_test, directory=None, x_col="path", 
                y_col=None, batch_size=1, shuffle=False, class_mode=None, target_size=(32,32))

**Training the model**

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID, epochs=50)

**Prediction on test data**

In [ ]:
y_pred = model.predict(test_generator)
y_pred_class = np.argmax(y_pred, axis=1)

**Calcualting KAPPA and Macro F1 score on test data**

In [ ]:
cohen = cohen_kappa_score(y_pred_class, y_test.astype('int'), weights='quadratic')
print("Quadratic Cohen kappa score of VGG model on test data is - %.3f" %cohen)

In [ ]:
print ('Classification Report : \n', classification_report(y_test.astype('int'), y_pred_class))
sns.heatmap(confusion_matrix(y_test.astype('int'), y_pred_class), annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('Actual class')
plt.xlabel('Predicted class')
plt.show()

**Reading Test data and predicting DR grades using trained model**

In [ ]:
img_path = list()
img_fullname = list()
img_name = list()
for dirname, _, filenames in os.walk('../input/aptos2019-blindness-detection/test_images/'):
    for filename in filenames:
        img_path.append(os.path.join(dirname, filename))
        temp = os.path.join(dirname, filename)
        temp = temp.split("/")[-1]
        img_fullname.append(temp)
        temp = temp.split(".")[0]
        img_name.append(str(temp))


In [ ]:
pre_df = pd.DataFrame()
pre_df['path'] = img_path
pre_df['fullname'] = img_fullname
pre_df['id_code'] = img_name

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator_2019=test_datagen.flow_from_dataframe(dataframe=pre_df, directory=None, x_col="path", 
                y_col=None, batch_size=1, shuffle=False, class_mode=None, target_size=(32,32))

In [ ]:
y_pred_2019 = model.predict(test_generator_2019)
y_pred_class_2019 = np.argmax(y_pred_2019, axis=1)

In [ ]:
filenames = list()
for i in test_generator_2019.filenames:
    temp = i.split('.')[-2]
    temp = temp.split('/')[-1]
    filenames.append(temp)

In [ ]:
submission_dense = pd.DataFrame()
submission_dense['id_code'] = filenames
submission_dense['diagnosis'] = y_pred_class_2019

In [ ]:
submission_dense

In [ ]:
submission_dense.to_csv('./submission.csv', index=False)